準備：`images.zip`に入っている画像をcolabに投げ込んでください

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

In [ ]:
# imgをrefに張り合わせることを考える
ref = cv2.imread("pano_ref.jpg") # ベースとなる画像（BGR）
src = cv2.imread("pano_src.jpg") # 変換する画像（BGR）

In [ ]:
# ratio test ありの場合
# https://docs.opencv.org/master/db/d27/tutorial_py_table_of_contents_feature2d.html

# ORB を使う場合。AKAZEならAKAZE_create、SIFTならSIFT_create
orb = cv2.ORB_create()

# 各画像に対するkeypointとdescriptorの計算
kp_ref, des_ref = orb.detectAndCompute(ref, None)
kp_src, des_src = orb.detectAndCompute(src, None)

# マッチング。ORBならHAMMING距離をつかうべき
matcher = cv2.BFMatcher(cv2.NORM_HAMMING) #, crossCheck=True) # knn matchのときはcross checkは使えない
matches = matcher.knnMatch(des_ref,des_src,k=2)
#matches = matcher.match(des_ref,des_src)

# ratio test を使ったロバスト化
good_matches = []

for m,n in matches:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

# 全対応を可視化
corr_disp = cv2.drawMatches(ref,kp_ref,src,kp_src,good_matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2_imshow(corr_disp) # 表示

# 対応点の登録とホモグラフィ行列の推定（w/ RANSAC）
pts_ref = np.float32([ kp_ref[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
pts_src = np.float32([ kp_src[m.trainIdx].pt  for m in good_matches]).reshape(-1,1,2)
H, mask = cv2.findHomography(pts_src, pts_ref, cv2.RANSAC,5.0)


In [ ]:
# 逆変換によるパノラマスティッチング（めんどいので最近傍法）

dst = np.zeros((src.shape[0],src.shape[1]*2,3), dtype=np.uint8) # 横幅2倍の画像を生成(縦src.shape[0],横src.shape[1],3ch)
dst[0:ref.shape[0], 0:ref.shape[1], :] = ref # 最初に、refの画素値を先に入れておく（部分配列の操作）。3次元目は色なので、そのまま(:)
H_inv = np.linalg.inv(H)  # 逆行列

for dst_y in range(dst.shape[0]):
  for dst_x in range(dst.shape[1]):
    dst_xyw = np.float32([dst_x, dst_y, 1]) # 同次座標
    src_xyw = H_inv.dot(dst_xyw)  # 変換
    src_x = src_xyw[0]/src_xyw[2] # 出力画像のX
    src_y = src_xyw[1]/src_xyw[2] # 出力画像のY

    if src_x < 1 or src_y < 1 or src_x > src.shape[1]-2 or src_y > src.shape[0]-2:  # 画像の外側を参照しないようにする
      continue

    dst[dst_y][dst_x] = src[int(src_y+0.5)][int(src_x+0.5)] # 最近傍法による補間


cv2_imshow(dst) # 表示

In [ ]:
# 色補正：refとdstの間の重複部分の画素値から最小二乗法

# refと変換後dstの重複領域をマスクとして計算
dst_ref = np.zeros((dst.shape[0],dst.shape[1],3), dtype=np.uint8)
dst_ref[0:ref.shape[0], 0:ref.shape[1], :] = ref  # 出力画像の座標における ref の画素値

mask_ref = np.zeros((dst.shape[0],dst.shape[1]), dtype=bool)
mask_ref[0:ref.shape[0], 0:ref.shape[1]] = True  # refが存在する場所がTrue

mask_src = np.zeros((dst.shape[0],dst.shape[1]), dtype=bool)
H_inv = np.linalg.inv(H)  # 逆行列

for dst_y in range(mask_src.shape[0]):
  for dst_x in range(mask_src.shape[1]):
    dst_xyw = np.float32([dst_x, dst_y, 1]) # 同次座標
    src_xyw = H_inv.dot(dst_xyw)  # 変換
    src_x = src_xyw[0]/src_xyw[2] # 出力画像のX
    src_y = src_xyw[1]/src_xyw[2] # 出力画像のY

    if src_x < 1 or src_y < 1 or src_x > src.shape[1]-2 or src_y > src.shape[0]-2:  # 画像の外側を参照しないようにする
      continue

    mask_src[dst_y][dst_x] = True # src画素が存在する場所がTrue

mask = mask_ref & mask_src  # 重複領域
cv2_imshow(mask*255)

# src, refそれぞれの重複領域の画素値をnumpy arrayとして作成（3*N、nは重複領域の面積）
# ヒント：img[mask]とすると、maskがTrueの要素を取り出せる
Y = dst_ref[mask].transpose() # refの色を3*Nの行列にする
X = dst[mask].transpose() # srcの色を3*Nの行列にする。dstは、srcを変換して上書きして作ったので、重複領域のdstの画素値はsrcからとってきたもの

# src -> refへの色補正行列の計算（線形最小二乗法を解く。講義スライド参照）
M = Y.dot(np.linalg.pinv(X))

# 変換行列による画素値の変換
src_pixels = dst[mask_src].transpose()
src_corrected = (M@src_pixels).transpose()
src_corrected = np.clip(src_corrected, 0, 255)
dst[mask_src] = src_corrected

# 表示
# サンプルの例では、周辺減光の影響で完全には色が一致しない。
# 重複領域をうまくアルファブレンディングすることで境目を目立たなくすることもできる。興味があれば試してみよう。
cv2_imshow(dst)

In [ ]:
# Appendix: OpenCVにおけるワーピング関数（デフォルトでは逆変換の後バイリニア補間）
# https://docs.opencv.org/master/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87
dst_opencv = cv2.warpPerspective(src, H, (src.shape[1]*2,src.shape[0]))

cv2_imshow(dst_opencv) # 表示